# Topic Modeling in Python using Latent Dirichlet Allocation (LDA)

Probabilistic topic models, such as LDA, are popular tools for text analysis, providing both a predictive and latent topic representation of the corpus. 
Here, we will focus on topic modeling approx 5000 tweets scrapped from twitter on Russia-Ukraine war.

### Model Implementation:
    1. Loading Data
    2. Data Cleaning
    3. Phrase Modeling: Bi-grams and Tri-grams
    4. Data Transformation: Corpus and Dictionary
    5. Base Model
    6. Hyper-parameter Tuning
    7. Final model

## Step 1: Loading Data

In [1]:
# Importing packages
import pandas as pd
import os

# Read the CSV file into a pandas DataFrame
df = pd.read_csv("../sentiment analysis using textblob/processed_tweets.csv")

# Print head
df.head()

,Datetime,Tweet Id,Text,Username,Subjectivity,Polarity,Analysis
0,2023-04-13 02:56:08+00:00,1646346501101174786,"Ukraine: In the video, a Ukrainian soldier wit...",WeaponsWarfare,0.321429,0.166667,Positive
1,2023-04-12 10:38:53+00:00,1646100569696632833,Ukraine: A very rare video of a Ukrainian BSEM...,WeaponsWarfare,1.000000,0.390000,Positive
2,2023-04-12 02:50:37+00:00,1645982725583585280,Ukraine: Very rare video - American🇺🇲 BATT UMG...,WeaponsWarfare,0.500000,0.195000,Positive
3,2023-04-07 18:01:33+00:00,1644400032362246145,"⚡️📢 Romanian media reports, that: ""Armenia ign...",ScourgeOfTengri,0.050000,-0.050000,Negative
4,2023-04-07 07:36:00+00:00,1644242607021740032,The Ukrainian counter-offensive is likely to b...,ukraine_world,1.000000,0.000000,Neutral


## Step 2: Data Cleaning

Since the goal of this analysis is to perform topic modeling, we will solely focus on the text data from each tweet and drop other metadata columns.

In [2]:
# Remove the columns
df = df.drop(columns=['Datetime', 'Tweet Id', 'Username', 
                              'Subjectivity', 'Polarity', 'Analysis'], axis=1)

# Print out the first rows of tweets
df.head()

,Text
0,"Ukraine: In the video, a Ukrainian soldier wit..."
1,Ukraine: A very rare video of a Ukrainian BSEM...
2,Ukraine: Very rare video - American🇺🇲 BATT UMG...
3,"⚡️📢 Romanian media reports, that: ""Armenia ign..."
4,The Ukrainian counter-offensive is likely to b...


### Remove punctuation/lower casing

Next, let’s perform a simple preprocessing on the content of text column to make them more amenable for analysis and reliable results. To do that, we’ll use a regular expression to remove any punctuation, and then lowercase the text

In [7]:
# Load the regular expression library
import re

# Remove punctuation
df['text_processed'] = df['Text'].map(lambda x: re.sub('[,\.!?]', '', x))

# Convert the titles to lowercase
df['text_processed'] = df['text_processed'].map(lambda x: x.lower())

# Print out the first rows of papers
df['text_processed'].head()

0    ukraine: in the video a ukrainian soldier with...
1    ukraine: a very rare video of a ukrainian bsem...
2    ukraine: very rare video - american🇺🇲 batt umg...
3    ⚡️📢 romanian media reports that: "armenia igno...
4    the ukrainian counter-offensive is likely to b...
Name: text_processed, dtype: object

### Tokenize words and further clean-up text

Let’s tokenize each sentence into a list of words, removing punctuations and unnecessary characters altogether.

In [8]:
import gensim
from gensim.utils import simple_preprocess

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data = df.text_processed.values.tolist()
data_words = list(sent_to_words(data))

print(data_words[:1][0][:30])

['ukraine', 'in', 'the', 'video', 'ukrainian', 'soldier', 'with', 'box', 'with', 'finnish', 'frag', 'hand', 'grenades', 'and', 'right', 'next', 'to', 'the', 'box', 'large', 'number', 'of', 'jva', 'fuses', 'russia', 'russia', 'kyiv', 'finland', 'ukrainewar', 'ukraina']


## Step 3: Phrase Modeling: Bigram and Trigram Models

Bigrams are two words frequently occurring together in the document. Trigrams are 3 words frequently occurring.

Gensim's Phrases model can build and implement the bigrams, trigrams, quadgrams and more. The two important arguments to Phrases are min_count and threshold.

The higher the values of these param, the harder it is for words to be combined.

In [9]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

### Remove Stopwords, Make Bigrams and Lemmatize

The phrase models are ready. Let’s define the functions to remove the stopwords, make trigrams and lemmatization and call them sequentially.

In [10]:
# NLTK Stop words
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

#### Let's call the functions in order.

In [ ]:
!python -m spacy download en_core_web_sm

In [13]:
import spacy

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1][0][:30])

['video', 'frag', 'hand', 'grenade', 'right', 'next', 'box', 'large', 'number', 'fuse']


## Step 4: Data transformation: Corpus and Dictionary

The two main inputs to the LDA topic model are the dictionary(id2word) and the corpus. Let’s create them.

In [14]:
import gensim.corpora as corpora

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1)]


## Step 5: Base Model

We have everything required to train the base LDA model. In addition to the corpus and dictionary, you need to provide the number of topics as well. Apart from that, alpha and eta are hyperparameters that affect sparsity of the topics. According to the Gensim docs, both defaults to 1.0/num_topics prior (we'll use default for the base model).

chunksize controls how many documents are processed at a time in the training algorithm. Increasing chunksize will speed up training, at least as long as the chunk of documents easily fit into memory.

passes controls how often we train the model on the entire corpus (set to 10). Another word for passes might be "epochs". iterations is somewhat technical, but essentially it controls how often we repeat a particular loop over each document. It is important to set the number of "passes" and "iterations" high enough.

In [15]:
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=10, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)

The above LDA model is built with 10 different topics where each topic is a combination of keywords and each keyword contributes a certain weightage to the topic.

You can see the keywords for each topic and the weightage(importance) of each keyword using lda_model.print_topics()

In [16]:
from pprint import pprint

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.021*"war" + 0.017*"country" + 0.016*"thank" + 0.014*"support" + '
  '0.012*"people" + 0.011*"help" + 0.011*"russian" + 0.010*"want" + '
  '0.009*"world" + 0.009*"ukraine"'),
 (1,
  '0.026*"ukraine" + 0.019*"say" + 0.018*"russian" + 0.016*"ukrainian" + '
  '0.013*"zelenskyy" + 0.012*"long" + 0.012*"day" + 0.012*"train" + '
  '0.010*"position" + 0.009*"take"'),
 (2,
  '0.038*"russian" + 0.025*"ukrainian" + 0.015*"attack" + 0.014*"kill" + '
  '0.013*"missile" + 0.013*"video" + 0.013*"force" + 0.012*"ukraine" + '
  '0.012*"military" + 0.010*"break"'),
 (3,
  '0.071*"standwithrussia" + 0.035*"situation" + 0.022*"operation" + '
  '0.021*"latest_defence" + 0.021*"intelligence_update" + 0.019*"special" + '
  '0.013*"good_night" + 0.012*"happen" + 0.010*"move" + 0.009*"many"'),
 (4,
  '0.027*"worldwar" + 0.023*"war" + 0.013*"see" + 0.011*"time" + 0.011*"big" + '
  '0.009*"leader" + 0.009*"world" + 0.008*"message" + 0.008*"people" + '
  '0.007*"arm"'),
 (5,
  '0.127*"worldwar" + 0.034*

#### Compute Model Perplexity and Coherence Score

Let's calculate the baseline coherence score

In [17]:
from gensim.models import CoherenceModel

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

Coherence Score:  0.388742523734377


## Step 6: Hyperparameter tuning

First, let's differentiate between model hyperparameters and model parameters :

Model hyperparameters can be thought of as settings for a machine learning algorithm that are tuned by the data scientist before training. Examples would be the number of trees in the random forest, or in our case, number of topics K

Model parameters can be thought of as what the model learns during training, such as the weights for each word in a given topic.

Now that we have the baseline coherence score for the default LDA model, let's perform a series of sensitivity tests to help determine the following model hyperparameters:

Number of Topics (K)
Dirichlet hyperparameter alpha: Document-Topic Density
Dirichlet hyperparameter beta: Word-Topic Density
We'll perform these tests in sequence, one parameter at a time by keeping others constant and run them over the two difference validation corpus sets. We'll use C_v as our choice of metric for performance comparison

In [18]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

Let's call the function, and iterate it over the range of topics, alpha, and beta parameter values

In [25]:
import numpy as np
import tqdm

grid = {}
grid['Validation_Set'] = {}

# Topics range
min_topics = 2
max_topics = 11
step_size = 1
topics_range = range(min_topics, max_topics, step_size)

# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')

# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')

# Validation sets
num_of_docs = len(corpus)
corpus_sets = [gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)), 
               corpus]

corpus_title = ['75% Corpus', '100% Corpus']

model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }

# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=(len(beta)*len(alpha)*len(topics_range)*len(corpus_title)))
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, 
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv('./lda_tuning_results.csv', index=False)
    pbar.close()

100%|██████████████████████████████████████████████████████████████████████████████| 540/540 [3:20:33<00:00, 22.28s/it]


## Step 7: Final Model

Based on external evaluation (Code to be added from Excel based analysis), let's train the final model with parameters yielding highest coherence score

In [26]:
num_topics = 8

lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=num_topics, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=0.01,
                                           eta=0.9)

In [27]:
from pprint import pprint

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.026*"worldwar" + 0.016*"war" + 0.010*"ukraine" + 0.009*"world" + '
  '0.008*"people" + 0.006*"country" + 0.006*"stop" + 0.005*"thank" + '
  '0.005*"standwithrussia" + 0.005*"go"'),
 (1,
  '0.013*"ukraine" + 0.011*"russian" + 0.008*"say" + 0.008*"ukrainian" + '
  '0.006*"day" + 0.006*"support" + 0.006*"war" + 0.004*"zelenskyy" + '
  '0.004*"take" + 0.004*"long"'),
 (2,
  '0.002*"gold" + 0.002*"eod" + 0.002*"anti" + 0.001*"team" + 0.001*"bronze" + '
  '0.001*"medal" + 0.001*"ukraineinvasion" + 0.001*"match" + 0.001*"pull" + '
  '0.001*"football"'),
 (3,
  '0.002*"happy" + 0.001*"corgicrew_guncontrolnow" + 0.001*"torture" + '
  '0.001*"agame" + 0.001*"cdnpoli" + 0.001*"crimeantatar" + 0.001*"english" + '
  '0.001*"watch" + 0.001*"trudeau" + 0.001*"laugh"'),
 (4,
  '0.010*"standwithrussia" + 0.009*"friend" + 0.004*"good_night" + '
  '0.004*"good_morning" + 0.001*"corgicrew_guncontrolnow" + 0.001*"big" + '
  '0.001*"fren" + 0.001*"dear_friend" + 0.001*"recent_update" + 0.001*"sorr

### Closing Notes

We started with understanding why evaluating the topic model is essential. Here, we reviewed existing methods and scratched the surface of topic coherence, along with the available coherence measures. Then we built a default LDA model using Gensim implementation to establish the baseline coherence score and reviewed practical ways to optimize the LDA hyperparameters.